In [47]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout 
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
import nltk
import sklearn
from sklearn.preprocessing import LabelEncoder
import random
import warnings
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
warnings.filterwarnings('ignore')
print('tensorflow version', tf.__version__)
 


tensorflow version 2.8.0


In [48]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [49]:
def cleanupSentences(sentence):
    strr=""
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in sentence_words]
    for i in sentence_words:
        strr=strr+i+" "
    return strr

In [50]:
lemmatizer= WordNetLemmatizer()
intents =json.loads(open('intents.json').read())
words=[]
classes=[]
Y= []
X= []
lettersToBeIgnored=['?','!', ',', '.',':']
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        wordList=nltk.word_tokenize(pattern)
        words.extend(wordList)
        X.append(cleanupSentences(pattern))
        Y.append(intent['tag'])
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in words if word not in lettersToBeIgnored]
words=sorted(set(words))
classes=sorted(set(classes))


In [51]:
enc = LabelEncoder()
enc.fit(Y)
Y = enc.transform(Y)

vocab_size = 10000
embedding_dim = 16
max_len = 20
trunc_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)


In [52]:
classes = len(classes)

model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(classes, activation='softmax'))

In [53]:
training_labels_final = np.array(Y)
EPOCHS = 400
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded, training_labels_final, epochs=EPOCHS)

Epoch 1/400
2/2 [==============================] - 1s 11ms/step - loss: 2.3975 - accuracy: 0.0286
Epoch 2/400
2/2 [==============================] - 0s 9ms/step - loss: 2.3929 - accuracy: 0.1429
Epoch 3/400
2/2 [==============================] - 0s 11ms/step - loss: 2.3895 - accuracy: 0.1429
Epoch 4/400
2/2 [==============================] - 0s 10ms/step - loss: 2.3860 - accuracy: 0.1429
Epoch 5/400
2/2 [==============================] - 0s 10ms/step - loss: 2.3824 - accuracy: 0.1429
Epoch 6/400
2/2 [==============================] - 0s 10ms/step - loss: 2.3785 - accuracy: 0.1429
Epoch 7/400
2/2 [==============================] - 0s 8ms/step - loss: 2.3745 - accuracy: 0.1429
Epoch 8/400
2/2 [==============================] - 0s 10ms/step - loss: 2.3702 - accuracy: 0.1429
Epoch 9/400
2/2 [==============================] - 0s 9ms/step - loss: 2.3653 - accuracy: 0.1429
Epoch 10/400
2/2 [==============================] - 0s 10ms/step - loss: 2.3607 - accuracy: 0.1429
Epoch 11/400
2/2 [====

In [ ]:
print("Hey there!, How can i help you?")
warnings.filterwarnings('ignore')
while True:
       
    string = input('')
    string=cleanupSentences(string)
    if string == 'exit': break
    result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]), truncating=trunc_type, maxlen=max_len))[0]
    result_index=np.argmax(result)
    if result[result_index] > 0.8:
        category = enc.inverse_transform([np.argmax(result)]) 
        for i in intents['intents']:
            if i['tag']==category:
                print(np.random.choice(i['responses']))
    else:
        print("Sorry I didn't catch that, could you try again please?")


Hey there!, How can i help you?
i want to make a safe environment for me and my family how can i do this
Maintain at least a 1-metre distance between yourself and others to reduce your risk of infection when they cough, sneeze or speak.  Maintain an even greater distance between yourself and others when indoors. The further away, the better.                Make wearing a mask a normal part of being around other people. The appropriate use, storage and cleaning or disposal are essential to make masks as effective as possible.                                                                                                            keep cleaning your hands                                                                                                             coughing into a bent elbow or tissue                                                                                                          avoide crowds
what are the right ways to wear a mask
Clean your hands before you put yo